In [1]:
# PANDAS IS FOR DATA WRANGLING
import pandas as pd

# IMPORT OUR LINEAR REGRESSION MODEL
from sklearn.linear_model import LinearRegression

# IMPORT OUR RANDOM FOREST REGERSSOR MODEL
from sklearn.ensemble import RandomForestRegressor


# IM LEAVING THERES HERE BECAUSE IM NICE
# METRICS HELP US SCORE OUR MODEL
from sklearn import metrics

# HELP US SPLIT OUR DATA INTO TESTING A TRAINING
from sklearn.model_selection import train_test_split

# HELPER FUNCTION WITH Z-SCORES TO REMOVE OUTLIERS
import scipy.stats as stats

%matplotlib inline

import pickle

### Load regression model 

In [2]:
filename = 'rf_regressor1.pkl'
model = pickle.load(open(filename, 'rb'))

#### Load category codes for non numerical cols 

In [3]:
city_cat_codes = pd.read_csv('city_cat_codes.csv')
stabbr_cat_codes = pd.read_csv('stabbr_cat_codes.csv')
acred_cat_codes = pd.read_csv('accred_cat_codes.csv')

In [4]:
# Read in basic inst data 
#Read in reduced institutional data 
na_vals = ['Na', 'NaN', 'PrivacySuppressed']
df = pd.read_csv('../data/Most-Recent-Cohorts-All-Data-Elements.csv', na_values=na_vals)

/home/ana/Projects/StudentDebtForecaster/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (9,1729,1743,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
selected_features = ['INSTNM', 'UNITID', 'CITY', 'STABBR', 'ACCREDAGENCY', 'CONTROL', 'COSTT4_A', 'COSTT4_P',
       'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'PFTFAC', 'PCTPELL', 'GRAD_DEBT_MDN']

In [6]:
df = df[selected_features]

In [7]:
df.head()

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN
0,Alabama A & M University,100654,Normal,AL,Southern Association of Colleges and Schools C...,1,22489.0,NaN,9744.0,18354.0,0.7411,0.7067,33375.0
1,University of Alabama at Birmingham,100663,Birmingham,AL,Southern Association of Colleges and Schools C...,1,24347.0,NaN,8568.0,19704.0,0.7766,0.3632,22500.0
2,Amridge University,100690,Montgomery,AL,Southern Association of Colleges and Schools C...,2,17680.0,NaN,6900.0,6900.0,1.0000,0.7673,27334.0
3,University of Alabama in Huntsville,100706,Huntsville,AL,Southern Association of Colleges and Schools C...,1,23441.0,NaN,10714.0,22362.0,0.6544,0.2698,21607.0
4,Alabama State University,100724,Montgomery,AL,Southern Association of Colleges and Schools C...,1,21476.0,NaN,11068.0,19396.0,0.5826,0.7448,32000.0


In [8]:
df[df.duplicated(['INSTNM'])]

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN
307,Glendale Community College,115001,Glendale,CA,Western Association of Schools and Colleges Ac...,1,11239.0,NaN,1175.0,6719.0,0.2899,0.3557,8000.0
880,Cannella School of Hair Design-Chicago,143491,Chicago,IL,National Accrediting Commission of Career Arts...,3,NaN,NaN,NaN,NaN,NaN,0.6429,NaN
1200,Highland Community College,155186,Highland,KS,Higher Learning Commission,1,11528.0,NaN,3077.0,3625.0,1.0000,0.3095,8598.0
1228,Southwestern College,155900,Winfield,KS,Higher Learning Commission,2,42702.0,NaN,30150.0,30150.0,1.0000,0.4115,23863.0
1481,Emmanuel College,165671,Boston,MA,New England Commission on Higher Education,2,55739.0,NaN,39804.0,39804.0,0.4724,0.2614,27000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6687,Design's School of Cosmetology,44125601,Pleasanton,CA,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,6333.0
6711,Make-up Designory,44936601,New York,NY,Accrediting Commission of Career Schools and C...,3,NaN,NaN,NaN,NaN,NaN,NaN,4962.0
6728,Cambridge College of Healthcare & Technology,45486501,Atlanta,GA,Accrediting Bureau of Health Education Schools,3,NaN,NaN,14827.0,NaN,NaN,NaN,28498.0
6729,Professional Hands Institute,45490801,Miami,FL,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df[df.INSTNM=='Glendale Community College']

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN
86,Glendale Community College,104708,Glendale,AZ,Higher Learning Commission,1,11843.0,NaN,2070.0,7854.0,1.0000,0.3292,8000.0
307,Glendale Community College,115001,Glendale,CA,Western Association of Schools and Colleges Ac...,1,11239.0,NaN,1175.0,6719.0,0.2899,0.3557,8000.0


In [26]:
df.duplicated(['UNITID']).sum()

0

In [25]:
df.duplicated(['INSTNM', 'CITY', 'STABBR']).sum()

16

In [42]:
selected_features

['INSTNM',
 'UNITID',
 'CITY',
 'STABBR',
 'ACCREDAGENCY',
 'CONTROL',
 'COSTT4_A',
 'COSTT4_P',
 'TUITIONFEE_IN',
 'TUITIONFEE_OUT',
 'PFTFAC',
 'PCTPELL',
 'GRAD_DEBT_MDN']

In [46]:
df.duplicated(['INSTNM', 'CITY', 'STABBR',  'CONTROL', 'COSTT4_A', 'COSTT4_P',
       'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'PFTFAC', 'PCTPELL', 'GRAD_DEBT_MDN']).sum()

1

In [28]:
df[df.duplicated(['INSTNM', 'CITY', 'STABBR'])]

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN,COSTT4_A_is_missing,COSTT4_P_is_missing,TUITIONFEE_IN_is_missing,TUITIONFEE_OUT_is_missing,PFTFAC_is_missing,PCTPELL_is_missing,GRAD_DEBT_MDN_is_missing
880,Cannella School of Hair Design-Chicago,143491,Chicago,IL,National Accrediting Commission of Career Arts...,3,22581.0,19589.0,11207.0,15275.0,0.5696,0.64290,13700.0,True,True,True,True,True,False,True
3318,Western Technical College,224679,El Paso,TX,Accrediting Commission of Career Schools and C...,3,22581.0,30502.0,11207.0,15275.0,0.6437,0.59520,14031.0,True,False,True,True,False,False,False
3341,International Business College-El Paso,225788,El Paso,TX,NaN,3,22581.0,24391.0,11207.0,15275.0,0.5696,0.83870,13254.0,True,False,True,True,True,False,False
4768,Interactive College of Technology,443696,Houston,TX,Council on Occupational Education,3,18225.0,19589.0,9530.0,9530.0,0.5696,0.77780,4094.0,False,True,False,False,True,False,False
5039,Trend Barber College,450711,Houston,TX,Council on Occupational Education,3,22581.0,13045.0,11207.0,15275.0,0.5696,0.49640,12500.0,True,False,True,True,True,False,False
5543,Trend Setters' Academy of Beauty Culture-Louis...,468705,Louisville,KY,NaN,3,22581.0,20859.0,11207.0,15275.0,0.5696,0.48000,5063.0,True,False,True,True,True,False,False
5578,Fortis College-Montgomery,475130,Montgomery,AL,NaN,3,27411.0,19589.0,14628.0,14628.0,0.5696,0.73990,9500.0,False,True,False,False,True,False,False
5612,Tulsa Welding School-Jacksonville,476063,Jacksonville,FL,Accrediting Commission of Career Schools and C...,3,22581.0,26405.0,11207.0,15275.0,0.5696,0.47920,9500.0,True,False,True,True,True,False,False
5773,Southern Careers Institute-San Antonio,482015,San Antonio,TX,Council on Occupational Education,3,22581.0,29964.0,11207.0,15275.0,0.5417,0.59540,8708.0,True,False,True,True,False,False,False
6309,Wade Gordon Hairdressing Academy,492591,Amarillo,TX,NaN,3,22581.0,27150.0,11207.0,15275.0,0.5696,0.24240,10359.0,True,False,True,True,True,False,False


In [24]:
# Fill nulls in X numeric columns with median, add column indicating if value was missing 
for label, content in df.items(): 
    if pd.api.types.is_numeric_dtype(content): 
        if pd.isnull(content).sum(): 
            #Add column that has true if row had missing value for the column 
            df[label+'_is_missing'] = pd.isnull(content) 
            # Fill missing values for median for the column 
            df[label] = content.fillna(content.median())

In [30]:
df[df.INSTNM == 'International Business College-El Paso']

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN,COSTT4_A_is_missing,COSTT4_P_is_missing,TUITIONFEE_IN_is_missing,TUITIONFEE_OUT_is_missing,PFTFAC_is_missing,PCTPELL_is_missing,GRAD_DEBT_MDN_is_missing
3340,International Business College-El Paso,225779,El Paso,TX,NaN,3,22581.0,23614.0,11207.0,15275.0,0.5696,0.7750,13254.0,True,False,True,True,True,False,False
3341,International Business College-El Paso,225788,El Paso,TX,NaN,3,22581.0,24391.0,11207.0,15275.0,0.5696,0.8387,13254.0,True,False,True,True,True,False,False


In [12]:
sample_input = {'INSTNM':'Glendale Community College', 'CITY': 'Glendale', 'STABBR': 'AZ'} 

In [14]:
condition = (df.INSTNM == sample_input['INSTNM']) & (df.CITY == sample_input['CITY']) & (df.STABBR == sample_input['STABBR'])
inst_data = df[condition] 

In [16]:
inst_data

,INSTNM,UNITID,CITY,STABBR,ACCREDAGENCY,CONTROL,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,PFTFAC,PCTPELL,GRAD_DEBT_MDN
86,Glendale Community College,104708,Glendale,AZ,Higher Learning Commission,1,11843.0,NaN,2070.0,7854.0,1.0,0.3292,8000.0
